## Closures: Anonymous Functions that Capture Their Environment

closures are anonymous functions you can save in a variable or pass as arguments to other functions.
- can be created in one place and then called elsewhere to evaluate it in a different context.
- closures can capture values from the scope in which they’re defined.
- closures in rust are like lambdas in python

closures capture a variable using `||` and are followed by an expression: `|_| <expr>`

### Capturing the Environment with Closures

Use an enum called `ShirtColor` with variants `Red` and `Blue`. We represent the company’s inventory with an
`Inventory` struct that has a field named `shirts` that contains a `Vec<ShirtColor>` representing the shirt
colors currently in stock. The method `giveaway` defined on `Inventory` gets the optional shirt color preference
of the free shirt winner, and returns the shirt color the person will get. Listing 13-1:

In [ ]:
#[derive(Debug, PartialEq, Copy, Clone)]
enum ShirtColor {
    Red,
    Blue,
}

struct Inventory {
    shirts: Vec<ShirtColor>,
}

impl Inventory {
    fn giveaway(&self, user_preference: Option<ShirtColor>) -> ShirtColor {
        user_preference.unwrap_or_else(|| self.most_stocked())  // closure used here, captures nothing
    }

    fn most_stocked(&self) -> ShirtColor {
        let mut num_red = 0;
        let mut num_blue = 0;

        for color in &self.shirts {
            match color {
                ShirtColor::Red => num_red += 1,
                ShirtColor::Blue => num_blue += 1,
            }
        }
        if num_red > num_blue {
            ShirtColor::Red
        } else {
            ShirtColor::Blue
        }
    }
}

fn main() {
    let store = Inventory {
        shirts: vec![ShirtColor::Blue, ShirtColor::Red, ShirtColor::Blue],
    };

    let user_pref1 = Some(ShirtColor::Red);
    let giveaway1 = store.giveaway(user_pref1);
    println!(
        "The user with preference {:?} gets {:?}",
        user_pref1, giveaway1
    );

    let user_pref2 = None;
    let giveaway2 = store.giveaway(user_pref2);
    println!(
        "The user with preference {:?} gets {:?}",
        user_pref2, giveaway2
    );
}

main()

### Closure Type Inference and Annotation

we can add type annotations if we want to increase explicitness and clarity at the cost of being more verbose than is strictly necessary

In [ ]:
use std::thread;
use std::time::Duration;

fn generate_workout(intensity: u32, random_number: u32) {
    let expensive_closure = |num: u32| -> u32 {
        println!("calculating slowly...");
        thread::sleep(Duration::from_secs(2));
        num
    };

    if intensity < 25 {
        println!("Today, do {} pushups!", expensive_closure(intensity));
        println!("Next, do {} situps!", expensive_closure(intensity));
    } else {
        if random_number == 3 {
            println!("Take a break today! Remember to stay hydrated!");
        } else {
            println!(
                "Today, run for {} minutes!",
                expensive_closure(intensity)
            );
        }
    }
}

fn main() {
    let simulated_user_specified_value = 10;
    let simulated_random_number = 7;

    generate_workout(simulated_user_specified_value, simulated_random_number);
}

main()

With type annotations added, the syntax of closures looks more similar to the syntax of functions. 
added some spaces to line up the relevant parts. This illustrates how closure syntax is similar to
function syntax except for the use of pipes and the amount of syntax that is optional:

```Rust
fn  add_one_v1   (x: u32) -> u32 { x + 1 }
let add_one_v2 = |x: u32| -> u32 { x + 1 };
let add_one_v3 = |x|             { x + 1 };
let add_one_v4 = |x|               x + 1  ;
```

For closure definitions, the compiler will infer one concrete type for each of their parameters and for their return value. Meaning that if you define a closure without any type annotations, and you use that closure with different types later, the compiler will throw a `mismatched types` error

What is the rationale for why Rust infers the types of arguments/returns for closures, but not for top-level functions (defined with `fn`)?
- Top-level functions can be part of a library's external interface, while closures cannot be directly exposed.

Rust permits pattern matching within closure arguments, including the use of underscores. For example, you could write:
```rust
let f = |_| (); // sometimes called the "toilet closure"
let s = String::from("Hello");
f(s);
```
What best describes the relationship between `f` and `s` here?
- `f` (toilet closure) causes `s` to be immediately dropped. 

### Capturing References or Moving Ownership

Closures can capture values from their environment in three ways, which directly map to the three ways a function can take a parameter:
- borrowing immutably
- borrowing mutably
- taking ownership

The closure will decide which of these to use based on what the body of the function does with the captured values.

define a closure that captures an immutable reference to the vector named list because it only needs an immutable reference to print the value:

In [5]:
fn main() {
    let list = vec![1, 2, 3];
    println!("Before defining closure: {list:?}");

    let only_borrows = || println!("From closure: {list:?}");

    println!("Before calling closure: {list:?}");
    only_borrows();
    println!("After calling closure: {list:?}");
}

main()

Before defining closure: [1, 2, 3]
Before calling closure: [1, 2, 3]
From closure: [1, 2, 3]
After calling closure: [1, 2, 3]


()

Change the closure body so that it adds an element to the list vector. The closure now captures a mutable reference:

In [2]:
fn main() {
    let mut list = vec![1, 2, 3];
    println!("Before defining closure: {list:?}");

    let mut borrows_mutably = || list.push(7);
    // println!("{:?}", list); // Error: cannot borrow `list` as immutable beause it is also borrowed as mutable
    borrows_mutably();
    println!("After calling closure: {list:?}");
}

main()

Before defining closure: [1, 2, 3]
After calling closure: [1, 2, 3, 7]


()

Can use the `move` keyword before parameter list to force the closure to take ownership of the values it uses in the environment, despite closure body not requiring ownership.
- This is mainly useful when passing a closure to a new thread to move the data so that it’s owne by the new thread:

In [7]:
use std::thread;

fn main() {
    let list = vec![1, 2, 3];
    println!("Before defining closure: {list:?}");

    thread::spawn(move || println!("From thread: {list:?}"))
        .join()
        .unwrap();
}

main()

Before defining closure: [1, 2, 3]
From thread: [1, 2, 3]


()

### Moving Captured Values Out of Closures and the `Fn` Traits

A closure body can do any of the following:
- move a captured value out of the closure
- mutate the captured value
- neither move nor mutate the value
- capture nothing from the environment to begin with.

The way a closure captures and handles values from the environment affects which traits the closure implements, and traits are how functions and structs specify the kinds of closures they can use. Closures will automatically implement at least one of the following three `Fn` traits (can also implement all of them) depending on how the closure's body handles the values:

1. `FnOnce` applies to closures that can be called once.
    - All closures implement at least this trait, because all closures can be called. A closure that moves captured values out of its body will only implement `FnOnce` and none of the other `Fn` traits, because it can only be called once.
2. `FnMut` applies to closures that don’t move captured values out of their body, but that might mutate the captured values.
    - These can be called more than once.
3. `Fn` applies to closures that don’t move captured values out of their body, don’t mutate captured values, and those that capture nothing from their environment.
    - These closures can be called more than once without mutating their environment, which is important in cases such as calling a closure multiple times concurrently.

### Closures Must Name Captured Lifetimes

When designing functions that accept or return closures, think about the lifetime of data captured by the closure. For example, a simple program that is supposed to return a closure that clones a string:

```rust
fn make_a_cloner(s_ref: &str) -> impl Fn() -> String {
    move || s_ref.to_string()
}
```
```
error[E0700]: hidden type for `impl Fn() -> String` captures lifetime that does not appear in bounds
 --> test.rs:2:5
  |
1 | fn make_a_cloner(s_ref: &str) -> impl Fn() -> String {
  |                         ---- hidden type `[closure@test.rs:2:5: 2:12]` captures the anonymous lifetime defined here
2 |     move || s_ref.to_string()
  |     ^^^^^^^^^^^^^^^^^^^^^^^^^
```
 the issue is that we need to tell Rust that the closure returned from `make_a_cloner` must not live longer than `s_ref`. We can do that explicitly using a lifetime parameter like so:
 
 ```rust
 //              vvvv         vv                             vvvv                
fn make_a_cloner<'a>(s_ref: &'a str) -> impl Fn() -> String + 'a {
    move || s_ref.to_string()
}
```
which basically says: `s_ref` is a string reference that lives for `'a`. Adding `+ 'a` to the return type’s trait bounds indicates that the closure must live no longer than `'a`. Therefore Rust deduces this function is now safe

The return type never mentioned this lifetime, so Rust could not deduce that `make_a_cloner` was safe. But if we explicitly say that the closure captures the lifetime of `s_ref`, then our function compiles.

can also use the lifetime elision rules to make the function type more concise. We can remove the `<'a>` generic so long as we keep an indicator that the returned closure depends on some lifetime, like this: 
```rust
fn make_a_cloner(s_ref: &str) -> impl Fn() -> String + '_ {
    move || s_ref.to_string()
}
```

In [8]:
use std::thread;

fn main() {
    let list = vec![1, 2, 3];
    println!("Before defining closure: {list:?}");

    thread::spawn(move || println!("From thread: {list:?}"))
        .join()
        .unwrap();
}